<a href="https://colab.research.google.com/github/Radityofajar/LearnHadoop/blob/main/FTDE_HW_Hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount the drive to google drive and save the UUD.txt on the Gdrive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Initiate Hadoop**

* Installing all the necessary JDK and library
* Adding all required property to the file

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
!update-alternatives --set jps /usr/lib/jvm/java-8-openjdk-amd64/bin/jps
!java -version

#Finding the default Java path
!readlink -f /usr/bin/java | sed "s:bin/java::"
!apt-get install openssh-server -qq > /dev/null
!service ssh start

!grep Port /etc/ssh/sshd_config

#Creating a new rsa key pair with empty password
!ssh-keygen -t rsa -P "" -f ~/.ssh/id_rsa <<< y

# See id_rsa.pub content
!more /root/.ssh/id_rsa.pub

#Copying the key to autorized keys
!cat $HOME/.ssh/id_rsa.pub > $HOME/.ssh/authorized_keys
#Changing the permissions on the key
!chmod 0600 ~/.ssh/authorized_keys

#Conneting with the local machine
!ssh -o StrictHostKeyChecking=no localhost uptime


#Downloading Hadoop 3.2.3
!wget -q https://archive.apache.org/dist/hadoop/common/hadoop-3.2.3/hadoop-3.2.3.tar.gz

#Untarring the file
!sudo tar -xzf hadoop-3.2.3.tar.gz
#Removing the tar file
!rm hadoop-3.2.3.tar.gz


#Copying the hadoop files to user/local
!cp -r hadoop-3.2.3/ /usr/local/
#-r copy directories recursively

#Adding JAVA_HOME directory to hadoop-env.sh file
!sed -i '/export JAVA_HOME=/a export JAVA_HOME=\/usr\/lib\/jvm\/java-8-openjdk-amd64' /usr/local/hadoop-3.2.3/etc/hadoop/hadoop-env.sh

import os
#Creating environment variables
#Creating Hadoop home variable

os.environ["HADOOP_HOME"] = "/usr/local/hadoop-3.2.3"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"
os.environ["PATH"] += f'{os.environ["JAVA_HOME"]}/bin:{os.environ["JRE_HOME"]}/bin:{os.environ["HADOOP_HOME"]}/sbin'

#Dowloading text example to use as input
#!wget -q https://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/101/101.txt

openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javac to provide /usr/bin/javac (javac) in manual mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/jps to provide /usr/bin/jps (jps) in manual mode
openjdk version "1.8.0_422"
OpenJDK Runtime Environment (build 1.8.0_422-8u422-b05-1~22.04-b05)
OpenJDK 64-Bit Server VM (build 25.422-b05, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64/jre/
 * Starting OpenBSD Secure Shell server sshd
   ...done.
#Port 22
#GatewayPorts no
Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh

In [3]:
#Adding required property to core-site.xlm file
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/core-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
  <property>
          <name>fs.defaultFS</name>
          <value>hdfs://localhost:9000</value>
          <description>Where HDFS NameNode can be found on the network</description>
  </property>
</configuration>
EOF

In [4]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/hdfs-site.xml
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>

</configuration>
EOF

In [5]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/mapred-site.xml
<?xml version="1.0"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>
<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
  </property>
  <property>
    <name>mapreduce.application.classpath</name>
    <value>$HADOOP_HOME/share/hadoop/mapreduce/*:$HADOOP_HOME/share/hadoop/mapreduce/lib/*</value>
  </property>

</configuration>
EOF

In [6]:
%%bash
cat <<EOF > $HADOOP_HOME/etc/hadoop/yarn-site.xml
<?xml version="1.0"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->
<configuration>
<property>
    <description>The hostname of the RM.</description>
    <name>yarn.resourcemanager.hostname</name>
    <value>localhost</value>
  </property>
  <property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
  </property>
  <property>
    <name>yarn.nodemanager.env-whitelist</name>
    <value>JAVA_HOME,HADOOP_COMMON_HOME,HADOOP_HDFS_HOME,HADOOP_CONF_DIR,CLASSPATH_PREPEND_DISTCACHE,HADOOP_YARN_HOME,HADOOP_HOME,PATH,LANG,TZ,HADOOP_MAPRED_HOME</value>
  </property>

<!-- Site specific YARN configuration properties -->

</configuration>
EOF

## **Formating the HDFS**


In [7]:
!$HADOOP_HOME/bin/hdfs namenode -format

#Creating other necessary enviroment variables before starting nodes
os.environ["HDFS_NAMENODE_USER"] = "root"
os.environ["HDFS_DATANODE_USER"] = "root"
os.environ["HDFS_SECONDARYNAMENODE_USER"] = "root"
os.environ["YARN_RESOURCEMANAGER_USER"] = "root"
os.environ["YARN_NODEMANAGER_USER"] = "root"

#Launching hdfs deamons
!$HADOOP_HOME/sbin/start-dfs.sh

#Launching yarn deamons
#nohup causes a process to ignore a SIGHUP signal
!nohup $HADOOP_HOME/sbin/start-yarn.sh

#Listing the running deamons
!jps

2024-08-17 15:20:32,190 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 1920d4359aa0/172.28.0.12
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.2.3
STARTUP_MSG:   classpath = /usr/local/hadoop-3.2.3/etc/hadoop:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/dnsjava-2.1.7.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/checker-qual-2.5.2.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/guava-27.0-jre.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/spotbugs-annotations-3.1.9.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/jetty-util-ajax-9.4.40.v20210413.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/avro-1.7.7.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/log4j-1.2.17.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/commons-logging-1.1.3.jar:/usr/local/hadoop-3.2.3/share/hadoop/common/lib/kerby-asn1-1.0.1.jar:/usr/local/hadoop-3.2.3/share/hado

In [8]:
#Report the basic file system information and statistics
!$HADOOP_HOME/bin/hdfs dfsadmin -report

Configured Capacity: 115658190848 (107.72 GB)
Present Capacity: 79997550592 (74.50 GB)
DFS Remaining: 79997526016 (74.50 GB)
DFS Used: 24576 (24 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:9866 (localhost)
Hostname: 1920d4359aa0
Decommission Status : Normal
Configured Capacity: 115658190848 (107.72 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 35643871232 (33.20 GB)
DFS Remaining: 79997517824 (74.50 GB)
DFS Used%: 0.00%
DFS Remaining%: 69.17%
Configured Cache Capacity: 0 (0 B)
Cache

In [9]:
#Exploring Hadoop utilities available
!ls $HADOOP_HOME/share/hadoop/tools/lib/

aliyun-java-sdk-core-4.5.10.jar      hadoop-gridmix-3.2.3.jar
aliyun-java-sdk-kms-2.11.0.jar	     hadoop-kafka-3.2.3.jar
aliyun-java-sdk-ram-3.1.0.jar	     hadoop-openstack-3.2.3.jar
aliyun-sdk-oss-3.13.0.jar	     hadoop-resourceestimator-3.2.3.jar
aws-java-sdk-bundle-1.11.901.jar     hadoop-rumen-3.2.3.jar
azure-data-lake-store-sdk-2.2.9.jar  hadoop-sls-3.2.3.jar
azure-keyvault-core-1.0.0.jar	     hadoop-streaming-3.2.3.jar
azure-storage-7.0.0.jar		     ini4j-0.5.4.jar
hadoop-aliyun-3.2.3.jar		     jdom2-2.0.6.jar
hadoop-archive-logs-3.2.3.jar	     kafka-clients-2.8.1.jar
hadoop-archives-3.2.3.jar	     lz4-java-1.7.1.jar
hadoop-aws-3.2.3.jar		     ojalgo-43.0.jar
hadoop-azure-3.2.3.jar		     opentracing-api-0.33.0.jar
hadoop-azure-datalake-3.2.3.jar      opentracing-noop-0.33.0.jar
hadoop-datajoin-3.2.3.jar	     opentracing-util-0.33.0.jar
hadoop-distcp-3.2.3.jar		     org.jacoco.agent-0.8.5-runtime.jar
hadoop-extras-3.2.3.jar		     wildfly-openssl-1.0.7.Final.jar
hadoop-fs2img-3.2.3.

# **Hadoop Streaming using Python**

1.   Make a directory files (similar with linux command)
2.   Copy file from local to HDFS
3.   Create mapper.py
4.   Create reducer.py
5.   Changing the permission of the files
6.   Running the MapReduce programs
7.   Check the output file
8.   Copy the output file to local


In [10]:
#Creating directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -mkdir /word_count_with_python

In [11]:
#Copying the file from local file system to Hadoop distributed file system (HDFS)
!$HADOOP_HOME/bin/hdfs dfs -put /content/drive/MyDrive/ColabNotebooks/FTDE-Hadoop/UUD.txt /word_count_with_python

In [12]:
# Check directory in HDFS
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python

Found 1 items
-rw-r--r--   1 root supergroup       1423 2024-08-17 15:21 /word_count_with_python/UUD.txt


In [13]:
%%writefile mapper.py

#!/usr/bin/env python

#'#!' is known as shebang and used for interpreting the script

# import sys because we need to read and write data to STDIN and STDOUT
import sys

# reading entire line from STDIN (standard input)
for line in sys.stdin:
  # to remove leading and trailing whitespace
  line = line.strip()
  # split the line into words, output data type list
  words = line.split()
  # delete [; , : . /]
  words = [word.replace('[', '').replace(';', '').replace(',', '').replace(':', '').replace('.', '').replace('/', '') for word in words]
  # lower all words
  words = [word.lower() for word in words]

  # we are looping over the words array and printing the word
  # with the count of 1 to the STDOUT
  for word in words:
    # write the results to STDOUT (standard output);
    # what we output here will be the input for the
    # Reduce step, i.e. the input for reducer.py
    print('%s\t%s' % (word, 1))

Writing mapper.py


In [14]:
%%writefile reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# read the entire line from STDIN
for line in sys.stdin:
  # remove leading and trailing whitespace
  line = line.strip()
  # splitting the data on the basis of tab we have provided in mapper.py
  word, count = line.split('\t', 1)
  # convert count (currently a string) to int
  try:
    count = int(count)
  except ValueError:
    # count was not a number, so silently
    # ignore/discard this line
    continue

  # this IF-switch only works because Hadoop sorts map output
  # by key (here: word) before it is passed to the reducer
  if current_word == word:
    current_count += count
  else:
    if current_word: #to not print current_word=None
      # write result to STDOUT
      print('%s\t%s' % (current_word, current_count))
    current_count = count
    current_word = word

# do not forget to output the last word if needed!
if current_word == word:
  print('%s\t%s' % (current_word, current_count))

Writing reducer.py


In [15]:
#Testing our MapReduce job locally (Hadoop does not participate here)
!cat '/content/drive/MyDrive/ColabNotebooks/FTDE-Hadoop/UUD.txt' | python mapper.py | sort -k1,1 | python reducer.py | head -10
#We apply sorting after the mapper because it is the default operation in MapReduce architecture

abadi	1
adil	2
allah	1
atas	2
bagi	1
bahwa	1
bangsa	3
bebas	1
beradab	1
berbahagia	1


In [16]:
#Changing the permissions of the files
!chmod 777 /content/mapper.py /content/reducer.py
#Setting 777 permissions to a file or directory means that it will be readable, writable and executable by all users

In [17]:
#Running MapReduce programs
!$HADOOP_HOME/bin/hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.2.3.jar \
  -input /word_count_with_python/UUD.txt \
  -output /word_count_with_python/output \
  -mapper "python /content/mapper.py" \
  -reducer "python /content/reducer.py"

packageJobJar: [/tmp/hadoop-unjar4216440069373083778/] [] /tmp/streamjob4864525865896900103.jar tmpDir=null
2024-08-17 15:21:28,448 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-08-17 15:21:28,775 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8032
2024-08-17 15:21:29,117 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1723908063797_0001
2024-08-17 15:21:29,488 INFO mapred.FileInputFormat: Total input files to process : 1
2024-08-17 15:21:30,034 INFO mapreduce.JobSubmitter: number of splits:2
2024-08-17 15:21:30,364 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1723908063797_0001
2024-08-17 15:21:30,366 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-08-17 15:21:30,653 INFO conf.Configuration: resource-types.xml not found
2024-08-17 15:21:30,653 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-08-17 15:21:31,1

In [18]:
!$HADOOP_HOME/bin/hdfs dfs -ls /word_count_with_python/output

Found 2 items
-rw-r--r--   1 root supergroup          0 2024-08-17 15:21 /word_count_with_python/output/_SUCCESS
-rw-r--r--   1 root supergroup       1141 2024-08-17 15:21 /word_count_with_python/output/part-00000


In [19]:

#Printing out first 50 lines
!$HADOOP_HOME/bin/hdfs dfs -cat /word_count_with_python/output/part-00000 | head -50

abadi	1
adil	2
allah	1
atas	2
bagi	1
bahwa	1
bangsa	3
bebas	1
beradab	1
berbahagia	1
berdasar	1
berdasarkan	1
berdaulat	1
berkat	1
berkedaulatan	1
berkehidupan	1
bersatu	1
dalam	3
dan	11
darah	1
daripada	1
dasar	1
dengan	6
depan	1
di	1
didorongkan	1
dihapuskan	1
dipimpin	1
disusunlah	1
dunia	2
esa	1
gerbang	1
hak	1
harus	1
hikmat	1
ialah	1
ikut	1
indonesia	12
ini	1
itu	4
karena	1
ke	1
keadilan	2
kebangsaan	2
kebijaksanaan	1
kehidupan	1
keinginan	1
kemanusiaan	1
kemerdekaan	5
kemerdekaannya	1


In [20]:
!$HADOOP_HOME/bin/hdfs dfs -copyToLocal /word_count_with_python/output/part-00000 /content/hdfs-wordcount.txt